In [5]:
from collections import defaultdict, Counter
from tqdm import tqdm

In [10]:
tag_analysis = Counter()
with open("../data/mSystemEntity/train/vi.txt") as f:
    for line in f:
        if not line.strip():
            continue
        tag = line.strip().split()[-1]

        tag_analysis.update([tag])

In [11]:
tag_analysis

Counter({'O': 8470203,
         'B-ORG': 109060,
         'I-ORG': 138113,
         'B-LOC': 139770,
         'B-PER': 42583,
         'I-PER': 30981,
         'I-LOC': 72679})

In [31]:
def check_entity_type_sample(sample):
    unique_tags = set([tag for tag in sample["tags"] if tag.startswith("B")])
    if len(unique_tags) == 1:
        return list(unique_tags)[0].split("-")[-1]
    elif not unique_tags:
        return "O"
    return "GENERAL"

def balance_data(data_file):
    sample_by_type = defaultdict(list)
    with open(data_file) as f:
        sample = {"words": [], "tags": []}
        for line in f:
            parts = line.strip().split()
            if not parts:
                sample_type = check_entity_type_sample(sample)
                sample_by_type[sample_type].append(sample)
                sample = {"words": [], "tags": []}
                continue
            sample["words"].append(parts[0])
            sample["tags"].append(parts[-1])
    
    min_num_sample = len(sample_by_type["O"])
    print("\n## BEFORE ##\n")
    for key, samples in sample_by_type.items():
        print(key, len(samples))
        if key in ["O", "GENERAL"]:
            continue
        if len(samples) < min_num_sample:
            min_num_sample = len(samples)
    print("\n## AFTER ##\n")
    total_samples = []
    for key, samples in sample_by_type.items():
        if key in ["O", "GENERAL"]:
            total_samples.extend(samples[:int(min_num_sample*2.4)])
            print(key, len(samples[:int(min_num_sample*2.4)]))
            continue
        if len(samples) > min_num_sample*1.2:
            total_samples.extend(samples[:int(min_num_sample*1.2)])
            print(key, len(samples[:int(min_num_sample*1.2)]))
        else:
            total_samples.extend(samples)
            print(key, len(samples))
    
    return total_samples

In [32]:
samples = balance_data("../data/mSystemEntity/train/ja.txt")


## BEFORE ##

O 1262165
ORG 97301
LOC 121031
PER 38583
GENERAL 2835

## AFTER ##

O 92599
ORG 46299
LOC 46299
PER 38583
GENERAL 2835
